# Foodify
---
  \
![Foodify](../img/Foodify_DarkGreen.png)
  
  \
  ¿Te has dado cuenta que una persona promedio gasta 1.5 horas a la semana en realizar la compra en el supermercado? Eso suponen 78 horas de media, lo que son más de 3 días perdidos al año en ir a comprar.
  
Pues bien, Foodify es una aplicación desarrollada en python que permite optimizar el proceso de compra de alimentos en los hogares y ahorrar todo ese tiempo para que lo puedas dedicar a otras cosas.
  
El propósito inicial de este proyecto es presentarlo como proyecto de fin de bootcamp de Data Analytics en Ironhack, por lo que durante el proceso de su construcción iré documentando todos los pasos que dé e iré añadiendo ficheros explicativos (jupyter notebooks) que vayan desglosando todo el proceso.
  
Dicho esto, empecemos...
  
# Estructura de desarrollo
---
  
El primer punto es establecer una estructura que me permita desarrollar el proyecto en su totalidad, por lo que lo voy a dividir en **n** partes en las que iré trabajando por separado. 
  
El objetivo base es establecer un producto mínimo viable que permita poner en producción la aplicación e ir mejorándola con el tiempo, añadiéndole nuevas funcionalidades hasta alcanzar un producto que considere adecuado para un uso estándar.
  
Por esa razón, el proyecto se va a dividir en:
  
1. Parte 1 - Creación de la arquitectura de la aplicación
2. Parte 2 - Realizar web scraping para obtener los precios más baratosde cada producto.
3. Parte 3 - Desarrollo de una API.  
4. Parte 4 - Utilizar machine learning para predecir la lista de la compra.  
5. Parte 5 - Integración con chatgpt para obtener recetas con lo que hay en la nevera.  
  
# Creación de la arquitectura de la aplicación
---

La aplicación se basa en tres pilares. 
  
  1. Base de datos de productos
  2. Productos en nevera o lista de la compra
  3. Precios de los productos

Para obtener una base de datos de los productos voy a realizar peticiones a una base de datos externa mediante la API de OpenFoodFacts.

OpenFoodFacts proporciona una base de datos de todos los productos que hay en el mercado, en la que mediante su código de barras o EAN, puedo acceder a diferentes valores del producto.

En este apartado, yo me he centrado en obtener los siguientes datos:

  1. EAN
  2. Nombre del producto
  3. Marca
  4. Tienda
  5. Imagen
  6. Score de nutrición

Para lograrlo y tras investigar observé que OpenFoodFact tiene una librería que permite hacer estas peticiones mediante métodos. Por lo que decido que es la manera más eficiente de gestionar esta parte y tras importarla realizo el código que me permite extraer estos valores.

# Base de datos en PostgreSQL
---
  
Una vez tengo el código para la extracción, el siguiente paso es crear la base de datos para almacenar los productos.
  
Para tener toda la información más ordenada he decidido establecer el flujo de datos de la siguietne manera:

1. Si se selecciona la opción de registro de compra:
    1. Escaneo de código de barras para registrar el producto comprado.
    2. Búsqueda de código de barras en mi tabla de Productos.
        a. Si está en la tabla, coge los datos y almacena un nuevo valor en la tabla Nevera
        b. Si no está en la tabla, porque es la primera vez que compro ese producto.
            1. Pide los datos a OpenFoodFacts
            2. Los almacena en la tabla Productos para tenerlos para la proxima vez
            3. Los almacena en la tabla Nevera
2. Si se selecciona la opción de producto gastado:
    1. Escaneo el código de barras para registrar el producto gastado.
    2. Busco el código de barras en mi tabla de Nevera.
    3. Añado el producto en la tabla de Lista de la compra.
    4. Elimino el producto de la tabla de Nevera.

Mediante estas opciones, soy capaz de gestionar el flujo del producto desde que lo escaneo por primera vez y lo introduzco en mi nevera, hata que lo he consumido o lo tiro a la basura.

![Flujo de nuevos productos](../img/Flujo_de_nuevo_producto.png)
  
# Obtención de precios
---
  
Una vez realizado este paso, el siguiente es obtener los precios en los diferentes supermercados de cada producto en cuestión.
  
Como no es necesario que los precios estén continuamente actualizándose, pero tampoco quiero que solo se actualicen cuando vaya a hacer la compra (ya que esto puede llevar un tiempo). He decidido que la obtención de datos se realice una vez al día a las 8:00, ya que por norma general los precios no cambian durante el día y de hacerlo lo hacen durante la noche.
  
Para poder obtener los diferentes precios de cada producto, he decidido realizarlo mediante web scrapping de cada supermercado en cuestión. De igual manera, dado que al final cada supermercado tiene un coste que hay que añadir por el envío, creo que no es producente incluir más de 5 supermercados, porque sino, los costes de envío serían demasiado elevados y no merecería la pena el ahorro conseguido en los productos.
  
Los supermercados de los que he decidido adquirir los precios son:

1. Mercadona
2. Día
3. Alcampo
4. Carrefour
  
Para adquirir los datos, y dado que son páginas estáticas, he decidido usar la librería BeautifulSoup ya que no necesita estar abriendo terminales como con selenium.
  
# Envío de datos a PostgreSQL
---
  
Para realizar el envío tanto de los datos de los productos a las diferentes tablas, como de los precios adquiridos mediante web scrapping, he decidido usar SQLAlchemy ya que me parece la manera más cómoda de gestionar todas las tablas y todo el proceso.
  
